In [1]:
# out of the box python modules
import sqlite3
import os
import pathlib
import math
import json

# Data wrangling stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler

# Evaluation metrics
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, max_error

# Neural network stuff
from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.models import TCNModel, RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score, rmse, smape
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood, QuantileRegression, ExponentialLikelihood, GammaLikelihood
from pytorch_lightning.callbacks import Callback, EarlyStopping

# import lightning.pytorch as pl
import pytorch_lightning as pl


import torch
from torch.nn import MSELoss

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)


rc('mathtext', default='regular')

%load_ext dotenv
%dotenv

DATABASE_PATH = os.environ['DATABASE_PATH']
MODEL_STORE_PATH = os.environ['MODEL_STORE']

def generate_torch_kwargs():
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    }

In [2]:
with sqlite3.connect(DATABASE_PATH) as conn:
    cur = conn.cursor()
    res = cur.execute("SELECT * FROM market_data where date >= '2020-09-01' order by ticker, date;")
    entries = res.fetchall()

full_dataset = pd.DataFrame(entries, columns=['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits'])
full_dataset['date'] = pd.to_datetime(full_dataset['date'])
full_dataset['avg_price'] = full_dataset[['open', 'high', 'low', 'close']].mean(axis=1)
full_dataset['paid_dividends'] = full_dataset['dividends'].replace(0, np.nan)
full_dataset['period'] = full_dataset.date.dt.to_period('Q')

In [3]:
class OptunaPruning(PyTorchLightningPruningCallback, pl.Callback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [4]:
def objective(trial):
    model_name = 'FirstTCN'
    callbacks = [OptunaPruning(trial, monitor="val_loss"), EarlyStopping("val_loss", min_delta=0.001, patience=3, verbose=True)]
    pl_trainer_kwargs = {"callbacks": callbacks}

    # set input_chunk_length, between 5 and 14 days
    days_in = trial.suggest_int("days_in", 20, 22)

    # set out_len, between 1 and 13 days (it has to be strictly shorter than in_len).
    days_out = trial.suggest_int("days_out", 19, days_in - 1)

    # Other hyperparameters
    kernel_size = trial.suggest_int("kernel_size", 8, 12)
    num_filters = trial.suggest_int("num_filters", 5, 10)
    weight_norm = trial.suggest_categorical("weight_norm", [False, True])
    likelihood = trial.suggest_categorical("likelihood", [None, GaussianLikelihood(), QuantileRegression(), ExponentialLikelihood()])
    dilation_base = trial.suggest_int("dilation_base", 2, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.25)
    lr = trial.suggest_float("lr", 0.01, 0.1, log=True)

    # build and train the TCN model with these hyper-parameters:
    model = TCNModel(
        input_chunk_length=days_in,
        output_chunk_length=days_out,
        dropout=dropout,
        dilation_base=dilation_base,
        weight_norm=weight_norm,
        kernel_size=kernel_size,
        likelihood=likelihood,
        num_filters=num_filters,
        pl_trainer_kwargs=pl_trainer_kwargs,
        optimizer_kwargs={"lr": lr},
        model_name=model_name,
        save_checkpoints=True,
        force_reset=True
    )
    
    model.fit(
        series=train_scaled,
        past_covariates=month_series,
        val_series=val_scaled,
        val_past_covariates=month_series,
        verbose=False
    )

    model = TCNModel.load_from_checkpoint(model_name=model_name, best=True)

    # Evaluate how good it is on the validation set
    preds = model.predict(series=train_scaled, n=len(val))
    preds_inverse_scaled = scaler.inverse_transform(preds)
    error = rmse(val, preds_inverse_scaled, verbose=True) # , n_jobs=-1
    # smape_val = np.mean(smapes)

    return error if error != np.nan else float("inf")

In [5]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [6]:
for ticker in full_dataset.ticker.unique():
    ticker_name = ticker.split('.')[0]
    sample = full_dataset.query(f"ticker == '{ticker}'")

    training_data_len = math.ceil(len(sample) * 0.8)
    train_data_date = sample.iloc[training_data_len, 1]

    ts = TimeSeries.from_dataframe(df=sample, time_col='date', value_cols='avg_price', freq='B')
    ts = fill_missing_values(ts, fill='auto')
    train, val = ts.split_after(0.7)

    scaler = Scaler()

    train_scaled = scaler.fit_transform(train)
    val_scaled = scaler.transform(val)
    ts_scaled = scaler.transform(ts)

    month_series = datetime_attribute_timeseries(ts, attribute="month", one_hot=True)


    ## Optimize hyper parameters
    study = optuna.create_study(direction="minimize")

    study.optimize(objective, timeout=7200, callbacks=[print_callback])

    # We could also have used a command as follows to limit the number of trials instead:
    # study.optimize(objective, n_trials=100, callbacks=[print_callback])

    # Finally, print the best value and best hyperparameters:
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

    with open(pathlib.Path(MODEL_STORE_PATH).joinpath(f'params/tcn_model_optim_params_{ticker_name}.json'), 'w') as f:
        json.dump(study.best_trial.params, f)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_contour(study, params=["lr", "num_filters"])

In [ ]:
plot_param_importances(study)

In [ ]:
study.best_trial.params